In [1]:

from quant_utils.db_conn import DB_CONN_JJTG_DATA
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def query_fund_alpha(trade_dt: str)->pd.DataFrame:
    query_sql = f"""
    SELECT
        END_DATE,
        TICKER_SYMBOL,
        `LEVEL`,
        INDICATOR,
        3M,
        6M,
        1Y,
        2Y,
        3Y
    FROM
        fund_derivatives_fund_alpha_performance 
    WHERE
        1 = 1 
        AND END_DATE = '{trade_dt}' 
    """
    df = DB_CONN_JJTG_DATA.exec_query(query_sql)
    df = df.set_index(["END_DATE", "TICKER_SYMBOL", "LEVEL", "INDICATOR"])
    df = df.stack(dropna=False).reset_index().rename(columns={0: 'VALUE', "level_4": "PERIOD"})
    return df

def query_fund_type(trade_dt: str)->pd.DataFrame:
    query_sql = f"""
    SELECT
        TICKER_SYMBOL,
        LEVEL_1,
        LEVEL_2,
        LEVEL_3
    FROM
        fund_type_own_temp 
    WHERE
        1 = 1 
        AND REPORT_DATE = (
        SELECT
            max( REPORT_DATE ) 
        FROM
            fund_type_own_temp 
        WHERE
            PUBLISH_DATE <= '{trade_dt}'
    )
    """
    df = DB_CONN_JJTG_DATA.exec_query(query_sql)
    df.set_index(["TICKER_SYMBOL"], inplace=True)
    df = df.stack().reset_index().rename(columns={"level_1": "LEVEL", 0: 'LEVEL_NAME'})
    return df

def query_fund_alpha_rank(trade_dt: str)->pd.DataFrame:
    alpha_perf = query_fund_type(trade_dt).merge(query_fund_alpha(trade_dt))
    alpha_perf["VALUE"] = alpha_perf.groupby(
        by=["LEVEL", "LEVEL_NAME", "INDICATOR", "PERIOD"]
    )["VALUE"].rank(pct=True, ascending=False, na_option='keep')

    condition = alpha_perf["INDICATOR"].isin(["MAXDD", "ANNUAL_VOL"])
    alpha_perf.loc[condition, "VALUE"] = (
        1 - alpha_perf.loc[condition, "VALUE"]
    )
    alpha_perf["VALUE"].fillna(0.5, inplace=True)
    return alpha_perf

def query_fund_alpha_score(trade_dt: str)->pd.DataFrame:
    alpha_perf_rank = query_fund_alpha_rank(trade_dt)
    indicator_weight = {
        "CUM_ALPHA": 0.35,
        "IR": 0.35,
        "MAXDD": 0.3,
        "ANNUAL_VOL": 0.0,
    }
    period_weight = {
        "3M": 0.2,
        "6M": 0.2,
        '1Y': 0.2,
        '2Y': 0.2,
        '3Y': 0.2,
    }
    alpha_perf_rank['INDICATOR_WEIGHT'] = alpha_perf_rank['INDICATOR'].map(indicator_weight).fillna(0)
    alpha_perf_rank['PERIOD_WEIGHT'] = alpha_perf_rank['PERIOD'].map(period_weight).fillna(0)
    alpha_perf_rank["VALUE"] = (
        alpha_perf_rank["VALUE"].mul(
            alpha_perf_rank["INDICATOR_WEIGHT"], 
            axis=0
        ).mul(
            alpha_perf_rank["PERIOD_WEIGHT"], 
            axis=0
        )
    )
    alpha_perf_score = (
        alpha_perf_rank
        .groupby(by=["TICKER_SYMBOL", "LEVEL", "LEVEL_NAME"])["VALUE"]
        .sum()
        .reset_index()
    )
    alpha_perf_score["VALUE"] = alpha_perf_score["VALUE"].replace(0, np.nan)
    return alpha_perf_score

def query_fund_alpha_score_rank(trade_dt: str)->pd.DataFrame:
    alpha_perf_score = query_fund_alpha_score(trade_dt)
    alpha_perf_score['RANK'] = (
        alpha_perf_score
        .groupby(by=["LEVEL", "LEVEL_NAME"])["VALUE"]
        .rank(pct=True, na_option='keep', ascending=False)*100
    )
    return alpha_perf_score

In [2]:
def query_fund_info(trade_dt)->pd.DataFrame:
    sql = f"""
    SELECT
        TICKER_SYMBOL,
        TICKER_SYMBOL_M,
        SEC_SHORT_NAME,
        IS_ILLIQUID,
        round(DATEDIFF( {trade_dt}, ESTABLISH_DATE ) / 365,2) AS '成立年限' 
    FROM
        fund_info 
    WHERE
        1 = 1 
        AND EXPIRE_DATE IS NULL 
        AND ESTABLISH_DATE IS NOT NULL
        and establish_date < {trade_dt}
    """
    return DB_CONN_JJTG_DATA.exec_query(sql)

def query_fund_asset(trade_dt: str)->pd.DataFrame:
    query_sql = f"""
    SELECT
        TICKER_SYMBOL,
        round( NET_ASSET / 100000000, 2 ) AS NET_ASSET 
    FROM
        fund_asset_own 
    WHERE
        1 = 1 
        AND REPORT_DATE = ( 
        SELECT max( REPORT_DATE ) 
        FROM fund_asset_own 
        WHERE 1 = 1 
        AND PUBLISH_DATE <= '{trade_dt}' )
    """
    return DB_CONN_JJTG_DATA.exec_query(query_sql)

In [3]:
trade_dt = "20241119"
columns_list = [
    'TICKER_SYMBOL', 
    'TICKER_SYMBOL_M', 
    'SEC_SHORT_NAME', 
    'IS_ILLIQUID', 
    '成立年限',
    'NET_ASSET',
    'LEVEL', 
    'LEVEL_NAME', 
    'VALUE', 
    'RANK',
]
df = query_fund_alpha_score_rank(trade_dt)
fund_info = query_fund_info(trade_dt)
fund_asset = query_fund_asset(trade_dt)
df = df.merge(fund_info).merge(fund_asset)
df = df[columns_list]

C:\Users\Wilcoxon\AppData\Local\Temp\ipykernel_24764\794304134.py:26: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df = df.stack(dropna=False).reset_index().rename(columns={0: 'VALUE', "level_4": "PERIOD"})
C:\Users\Wilcoxon\AppData\Local\Temp\ipykernel_24764\794304134.py:64: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


In [4]:
df.to_excel("d:/基金超额筛选.xlsx", index=False)